In [50]:
import pandas as pd
import numpy as np
from transformers import pipeline
import spacy
from spacy import displacy

In [35]:
df = pd.read_csv("../data/tesla_solar_roof_comments.csv")

In [36]:
df.shape

(25832, 9)

In [37]:
sdf = df[["authorDisplayName","comment"]]
sdf.head()

,authorDisplayName,comment
0,@mkbhd,So it appears Tesla found a bug in their app a...
1,@merojeff,Why didn’t you factor any SREC’s into your cal...
2,@sreejithprakash13s36,Straight from Twitter link after you posted th...
3,@hbjigcc,"Yes, you cannot produce 350 kWh of energy in a..."
4,@nyemartin5737,I may have missed it but don't you sell electr...


In [38]:
model_path="cardiffnlp/twitter-roberta-base-sentiment-latest"
model = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/dinyad/anaconda3/envs/tf/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `F

In [39]:
print(model("Hi, how are you?"))
print(model("Fucking shit!"))
print(model("What a nice day!"))

[{'label': 'neutral', 'score': 0.5607288479804993}]
[{'label': 'negative', 'score': 0.9157103300094604}]
[{'label': 'positive', 'score': 0.9802619218826294}]


In [40]:
def get_sentitment(text):
    try:
        res = model(text)
        return res[0].get("label")
    except:
        print("----START\n",text,"\n____END\n")
        return "unknown"

In [41]:
get_sentitment("Hi there!")

'positive'

In [43]:
sdf["sentiment"] = np.vectorize(get_sentitment)(sdf.comment)

----START
 I may not understand this correctly but something about your kWh usage for the year seems off.

You said you used 54,380 kWh in one year. That's an average of 4,532 kWh per month. The average family uses 1,000 kWh per month. This includes lights, AC, other various electronics. A Tesla to charge from 0% battery to 100% battery roughly uses 50 kWh to charge fully per day. If you charge 50 kWh everyday (You said you only use 20% battery in the winter and that's at worst) that would be 1,500 kWh per month. You didn't even charge your car for several days at a time. You also said you don't use electric heating so we can exclude that.

(Your) 4,532 kWh - (Avg) 1,000 kWh - (Tesla Car) 1,500 kWh = 2,032 kWh
That means you have an unaccounted 2,032 kWh of use per month on average. (I would say this number is closer to ~3,000 kWh with average car charging kWh)

WTF are you doing to use that much extra energy?! You are using an additional 2x the average family household kWh of energy p

/var/folders/jl/v49tv3qj63xczq_mz3f2wmqr0000gn/T/ipykernel_26946/1798462846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf["sentiment"] = np.vectorize(get_sentitment)(sdf.comment)


In [44]:
sdf.head(20)

,authorDisplayName,comment,sentiment
0,@mkbhd,So it appears Tesla found a bug in their app a...,negative
1,@merojeff,Why didn’t you factor any SREC’s into your cal...,negative
2,@sreejithprakash13s36,Straight from Twitter link after you posted th...,neutral
3,@hbjigcc,"Yes, you cannot produce 350 kWh of energy in a...",neutral
4,@nyemartin5737,I may have missed it but don't you sell electr...,neutral
5,@ntingk,I really liked the Tesla software. They are gr...,positive
6,@kcutoob,I don't understand. Did you use the doubled us...,neutral
7,@Lucidnaire,Dude none of this makes sense. It sensible mak...,negative
8,@kcutoob,@@Lucidnaire You are 100% correct. We need an...,negative
9,@Lucidnaire,@@kcutoob we 100% deserve the true story. Like...,negative


In [45]:
sdf["sentiment"].value_counts()

sentiment
neutral     10417
positive     7718
negative     7650
unknown        47
Name: count, dtype: int64

In [46]:
sdf.to_csv("./saved/comments_with_sentitment.csv", index=False)